In [5]:
#---------------------获取线元嵌入-------------------------------
from gettext import npgettext
from msilib import sequence
from transformers import BertForMaskedLM, BertTokenizer,BertModel  
import torch,os  
import numpy as np
# 设置要使用的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 显式指定使用哪个GPU
torch.cuda.set_device(0)  
# 加载预训练的BERT模型和分词器  
ppath = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\data\custom_tokenizer0318\vocabContourW0512.txt'
model_path = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\result\Cps30epoch40Len128LR1e-4Bt256warm1000_H512L8A8\checkpoint-63000'  # 你的模型保存路径  
#待输入到BERT的句子txt
input_Sequence_txtpath = r'G:\BT\data\3downStreamTask\LandformUnit\sequenceCorpusJoinJXK.txt'
#所需要获取嵌入的线元（TagID、以及在上边句子txt的行列位置）信息
input_LineUnitPositionInSequence_txtpath = r'G:\BT\data\3downStreamTask\LandformUnit\LineUnitPositionInSequence.txt'
#输出线元TagID及其嵌入
outTagID_Embedding_txtpath=r'G:\BT\data\3downStreamTask\LandformUnit\TagID_Embedding.txt'
#输出线元TagID及其word
outTagID_word_txtpath=r'G:\BT\data\3downStreamTask\LandformUnit\TagID_word.txt'

LineUnitPositionInSequence_dic={}#存储各行的各个 线元TagID的位置
LineUnitPositionIn_1Sequence_dic={}#存储一行中各个 线元TagID的位置
outTagID_Embedding_dic={}#存储每个TagID及其嵌入
#用于记录：
outTagID_word_dic={}#存储每个TagID及其对应序列的word
positionInMax_len=[]#存储目标线元在序列的第127各位置 的那些行
positionInMin_len=[]#存储目标线元在序列的开头位置 的那些行
max_len=128

#加载待获取嵌入的线元信息
input_LineUnitPositionInSequence=np.loadtxt(input_LineUnitPositionInSequence_txtpath,dtype=str,delimiter=' ')
for i in range(len(input_LineUnitPositionInSequence)):  
    tagID,row,col=input_LineUnitPositionInSequence[i]
    row,col=int(row),int(col)
    if col==(max_len-1) or col==(max_len-2):
        positionInMax_len.append(row)
    if col==0 or col==1:
        positionInMin_len.append(row)
    if row not in LineUnitPositionInSequence_dic:  
        LineUnitPositionIn_1Sequence_dic={}
        LineUnitPositionIn_1Sequence_dic[tagID]=col
        LineUnitPositionInSequence_dic[row]=LineUnitPositionIn_1Sequence_dic
    else:
        LineUnitPositionIn_1Sequence_dic[tagID]=col
        LineUnitPositionInSequence_dic[row]=LineUnitPositionIn_1Sequence_dic
        
# 加载tokenizer（分词器）  
# tokenizer = BertTokenizer.from_pretrained(ppath)    #加载预训练模型的词汇表
tokenizer = BertTokenizer(vocab_file=ppath, unk_token="[UNK]", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]", mask_token="[MASK]")
tokenizer.model_max_length = max_len
# 加载模型  
model = BertForMaskedLM.from_pretrained(model_path,output_hidden_states=True)  
  
# 如果有GPU，则将模型移动到GPU上  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model.to(device)  
  
# 遍历待输入到BERT的句子txt
row_num=0
with open(input_Sequence_txtpath, 'r', encoding='utf-8') as f:
    for line in f:
        isInMax_lenPosition=False
        sequence = line.strip('\n')
        tokens=tokenizer.tokenize(sequence)
        if len(tokens)>(max_len-2):  # 如果句子长度超过最大长度，则截断
            if row_num in positionInMax_len:
                isInMax_lenPosition=True
                tokens=tokens[2:max_len]
                # if row_num not in positionInMin_len:
                #     tokens=tokens[2:max_len]
                # else:#此时，该行的开头和结尾单词都在JXK内【未解决0520】
            else:
                tokens=tokens[:max_len-2]
        # print(tokens)
        tokens=['[CLS]']+tokens+['[SEP]']
        # print(tokens)
        attention_mask=[1 if i!='[PAD]' else 0 for i in tokens]
        # print(attention_mask)
        token_ids=tokenizer.convert_tokens_to_ids(tokens)
        # print(token_ids)
        token_ids=torch.tensor(token_ids).unsqueeze(0)
        attention_mask=torch.tensor(attention_mask).unsqueeze(0)
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)

        # 获取模型的嵌入输出
        with torch.no_grad():
            outputs = model(token_ids, attention_mask=attention_mask)
        hidden_rep = outputs.hidden_states
        tokens_embedding=hidden_rep[-1][0]
        LineUnitPositionIn_1Sequence_dic_temp=LineUnitPositionInSequence_dic[row_num]
        if isInMax_lenPosition:#如果该行存在线元在第max_len位,tokens组成是:['[CLS]']+tokens[2:max_len-1]+['[SEP]']
            for key,value in LineUnitPositionIn_1Sequence_dic_temp.items():
                outTagID_Embedding_dic[key]=tokens_embedding[value-1]#减1是因为tagID在原始tokens的位置-1（如3-1）对应于目前tokens的位置（如2），例如原始tokens[token0,token1,token2],现在tokens[[Cls],token2]
                outTagID_word_dic[key]=tokens[value-1]            
        else:    #如果该行  不 存在线元在第max_len位,tokens组成是：['[CLS]']+tokens[:max_len-2]+['[SEP]']
            for key,value in LineUnitPositionIn_1Sequence_dic_temp.items():
                outTagID_Embedding_dic[key]=tokens_embedding[value+1]#加1是因为BERT中句子以[Cls]开头，0是[Cls].例如原始tokens[token0,token1,token2,...token127],现在tokens[[Cls],token0,token1,...token125]
                outTagID_word_dic[key]=tokens[value+1]
        row_num=row_num+1
        # outTagID_Embedding_dic
        # predic_token = outputs.logits
        # print('hidden_rep[-1].shape')
        # print(hidden_rep[-1].shape)#torch.Size([batchsize, sequenceLength,hidden_dims])最后一个隐藏层中各个词的嵌入
        # print('hidden_rep[0]')
        # print(hidden_rep[0])
                
        # print('predic_token.shape')
        # print(predic_token.shape)#torch.Size([batchsize, sequenceLength, vocabSize]) 每个token被预测为词汇表中词的概率，未经softmax处理。
        # print(predic_token[0][0])
        # print(predic_token[0][1])
#将 outTagID_Embedding_dic 写入到txt中outTagID_Embedding_txtpath
with open (outTagID_Embedding_txtpath,'w') as file:
    for key,value in outTagID_Embedding_dic.items():
        # 将张量转为CPU上的numpy数组，以便更容易地转换为字符串
        numpy_array = value.cpu().numpy()
        # 使用numpy的savetxt方法将数组转换为字符串（这里我们直接构造字符串）
        tensor_str = ' '.join(map(str, numpy_array.flatten().tolist()))
        # 写入键和值（张量转换后的字符串形式）
        file.write(f'{key}: {tensor_str}\n')    
with open (outTagID_word_txtpath,'w') as file:
    for key,value in outTagID_word_dic.items():     
        file.write(f'{key}: {value}\n')
print('完成：获取嵌入')  

完成：获取嵌入


In [13]:
#---------------------获取序列嵌入-------------------------------
from gettext import npgettext
from msilib import sequence
from transformers import BertForMaskedLM, BertTokenizer,BertModel  
import torch,os  
import numpy as np
# 设置要使用的GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# 显式指定使用哪个GPU
torch.cuda.set_device(0)  
# 加载预训练的BERT模型和分词器  
ppath = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\data\custom_tokenizer0318\vocabContourW0512.txt'
model_path = r'G:\BT\program\ContourBERT\dataTokenCls_SentencenSegmentation\result\Cps30epoch40Len128LR1e-4Bt256warm1000_H512L8A8\checkpoint-63000'  # 你的模型保存路径  
#待输入到BERT的句子txt
input_Sequence_txtDiretory=r'G:\BT\data\3downStreamTask\MorphologicalClustering\3Corpus126_0606'
# input_Sequence_txtpath = r'G:\BT\data\3downStreamTask\MorphologicalClustering\3Corpus126_0606\LineUnit_平滑平缓型.txt'
out_Embedding_directory=r'G:\BT\data\3downStreamTask\MorphologicalClustering\4EmbeddingOfSequence'
out_sequence_embedding_lst=[]

# 加载tokenizer（分词器）  
# tokenizer = BertTokenizer.from_pretrained(ppath)    #加载预训练模型的词汇表
tokenizer = BertTokenizer(vocab_file=ppath, unk_token="[UNK]", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]", mask_token="[MASK]")
tokenizer.model_max_length = max_len
# 加载模型  
model = BertForMaskedLM.from_pretrained(model_path,output_hidden_states=True)  
  
# 如果有GPU，则将模型移动到GPU上  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model.to(device)  
  
# 遍历待输入到BERT的句子txt
# 遍历目标目录及其所有子目录  
for root, dirs, files in os.walk(input_Sequence_txtDiretory):  
    for input_Sequence_txtName in files:  
        if not input_Sequence_txtName.endswith(".txt"):
            print(input_Sequence_txtName)
            continue
        out_sequence_embedding_lst=[]
        input_Sequence_txtpath=os.path.join(input_Sequence_txtDiretory,input_Sequence_txtName)
        with open(input_Sequence_txtpath, 'r', encoding='utf-8') as f:
            out_Embedding_path=os.path.join(out_Embedding_directory,input_Sequence_txtName)
            if os.path.exists(out_Embedding_path): 
                # 如果文件存在，则删除它  
                try:  
                    os.remove(out_Embedding_path)  
                    print(f"文件 {out_Embedding_path} 已成功删除。")  
                except OSError as e:  
                    print(f"删除文件 {out_Embedding_path} 时出错: {e.strerror}")             
            for line in f:
                # isInMax_lenPosition=False
                sequence = line.strip('\n')
                tokens=tokenizer.tokenize(sequence)
                tokens=['[CLS]']+tokens+['[SEP]']
                # print(tokens)
                attention_mask=[1 if i!='[PAD]' else 0 for i in tokens]
                # print(attention_mask)
                token_ids=tokenizer.convert_tokens_to_ids(tokens)
                # print(token_ids)
                token_ids=torch.tensor(token_ids).unsqueeze(0)
                attention_mask=torch.tensor(attention_mask).unsqueeze(0)
                token_ids = token_ids.to(device)
                attention_mask = attention_mask.to(device)

                # 获取模型的嵌入输出
                with torch.no_grad():
                    outputs = model(token_ids, attention_mask=attention_mask)
                hidden_rep = outputs.hidden_states
                tokens_embedding=hidden_rep[-1][0]
                sequence_embedding=tokens_embedding[0].cpu().tolist()
                out_sequence_embedding_lst.append(sequence_embedding)
        
        np.savetxt(out_Embedding_path,out_sequence_embedding_lst)  
        print('完成：获取嵌入'+input_Sequence_txtpath)  
          

文件 G:\BT\data\3downStreamTask\MorphologicalClustering\4EmbeddingOfSequence\LineUnit_平滑平缓型.txt 已成功删除。
完成：获取嵌入G:\BT\data\3downStreamTask\MorphologicalClustering\3Corpus126_0606\LineUnit_平滑平缓型.txt
文件 G:\BT\data\3downStreamTask\MorphologicalClustering\4EmbeddingOfSequence\LineUnit_平滑陡峭型.txt 已成功删除。
完成：获取嵌入G:\BT\data\3downStreamTask\MorphologicalClustering\3Corpus126_0606\LineUnit_平滑陡峭型.txt
文件 G:\BT\data\3downStreamTask\MorphologicalClustering\4EmbeddingOfSequence\LineUnit_曲折平缓型.txt 已成功删除。
完成：获取嵌入G:\BT\data\3downStreamTask\MorphologicalClustering\3Corpus126_0606\LineUnit_曲折平缓型.txt
文件 G:\BT\data\3downStreamTask\MorphologicalClustering\4EmbeddingOfSequence\LineUnit_曲折陡峭型.txt 已成功删除。
完成：获取嵌入G:\BT\data\3downStreamTask\MorphologicalClustering\3Corpus126_0606\LineUnit_曲折陡峭型.txt
